## mimlsvm

mi = mulit-instance
ml = multi-label
svm = svm

As described in Shen et al 2009: http://ieeexplore.ieee.org/document/5346261/

> Should we use SVM-struct instead? https://github.com/pystruct/pystruct


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from hausdorff import hausdorff

from nltk import TextTilingTokenizer
from scipy.spatial.distance  import directed_hausdorff, pdist

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [3]:
%aimport src.data.movielens_20m_imdb
%aimport src.helpers.labels,src.helpers.neighbours, src.helpers.segments
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics

In [4]:
from src.data.movielens_20m_imdb import load_df_or_get_from_cache
from src.helpers.labels import truncate_labels
from src.helpers.neighbours import get_predicted_labels_from_neighbours
from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments

from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.clusters import k_medoids

In [5]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/movielens-ovr-linear-svc-calibrated/")
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/movielens-ml20m-imdb/")
PATH_TO_PROCESSED_FILE = os.path.abspath('../../../data/processed/movielens-20m-imdb-tags-and-synopses-2017-12-20.csv')

# CONFIGS

SEED= 42
MAX_NB_WORDS = 300
NB_DOCS=0.2
MIN_TAG_DF=10

# preprocessing

VECTORIZER_NORM = ['l1','l2',None]

W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

# calculating medoids
NORMALIZATION = ['standard','minmax',None] # not specified in the paper
SAMPLE_TO_NB_MEDOIDS_RATIO = [0.2,0.3,0.4] # not specified in the paper, but taken from MIMLSVM canonical implementation

# classification
SVM_KERNEL=['rbf','linear','poly'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_GAMMA=[0.1,0.2,0.3,0.5,1.0,'auto'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= [0.001,0.01,0.1,1]# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=[3,4,5]

In [6]:
docs_df = load_df_or_get_from_cache(PATH_TO_PROCESSED_FILE,INTERIM_DATA_ROOT)

In [7]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
# remove this for production
docs_df = sample_rows(docs_df,NB_DOCS)

In [9]:
docs_df['sentences'] = docs_df['synopsis'].map(lambda row: sentence_tokenizer.tokenize(row))

In [10]:
%%time
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents))

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.43 ms


In [11]:
docs_df.head()

,movie_id,title,synopsis,tags,num_tags,sentences,num_sentences
0,5154,"Arena, The (a.k.a. Naked Warriors) (1974)",In ancient Rome audiences have grown tired of ...,"rome,nudity-full-frontal,sword-and-sandal,nudi...",9,[In ancient Rome audiences have grown tired of...,3
1,2141,"American Tail, An (1986)",The film begins in the small Russian village o...,"can't-remember,animated,immigrants,cartoon",4,[The film begins in the small Russian village ...,156
2,32385,"Toolbox Murders, The (1978)",A person is driving around Los Angeles at nigh...,"remade,betamax",2,[A person is driving around Los Angeles at nig...,73
3,124224,Whistling in Brooklyn (1943),"Red Skelton filmed a series of these ""whistlin...",bd-r,1,"[Red Skelton filmed a series of these ""whistli...",7
4,8781,"Manchurian Candidate, The (2004)",This is a remake of the 1962 thriller of soldi...,"bibliothek,ei-muista,brainwashing,assassin,cor...",16,[This is a remake of the 1962 thriller of sold...,10


In [12]:
docs_df.iloc[0]['sentences']

['In ancient Rome audiences have grown tired of viewing gladiators fight to the death.',
 'Something new is needed--women!',
 '(Josiah Howard, "Blaxploitation cinema: The Essential Reference Gudie.")']

In [13]:
docs_df.describe()

,movie_id,num_tags,num_sentences
count,1342.000000,1342.000000,1342.000000
mean,42235.264531,11.923249,57.635618
std,39860.198123,13.463600,77.233093
min,7.000000,1.000000,1.000000
25%,4166.750000,3.000000,13.000000
50%,33101.000000,7.000000,36.000000
75%,78040.500000,16.000000,69.000000
max,128632.000000,101.000000,1136.000000


In [14]:
tok = TextTilingTokenizer(w=W, k=K)

In [15]:
def extract_segments(candidates):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [16]:
%%time
docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))

CPU times: user 10min 22s, sys: 0 ns, total: 10min 22s
Wall time: 10min 21s


In [17]:
docs_df['segments'][0][:3]

['In ancient Rome audiences have grown tired of viewing gladiators fight to the death.',
 'Something new is needed--women!',
 '(Josiah Howard, "Blaxploitation cinema: The Essential Reference Gudie.")']

In [18]:
segments = docs_df['segments'].values
documents = docs_df['synopsis'].values
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

In [19]:
# remove for production
labels = truncate_labels(labels,MIN_TAG_DF)

In [20]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [21]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)

In [22]:
# segments, documents and labelsets are defined outside of the parameterGrid loop
# because they're the same for every configuration    
segments_train, segments_test, documents_train, documents_test, Y_train, Y_test = train_test_split(segments,
                                                                                               documents,
                                                                                               labels,
                                                                                               test_size=0.15)

print('total number of train documents: {}'.format(len(documents_train)))
print('total number of validation documents: {}'.format(len(documents_test)))

total number of train documents: 1140
total number of validation documents: 202


In [23]:
mlb = MultiLabelBinarizer()


# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labels)

Y_train = mlb.transform(Y_train)
Y_test = mlb.transform(Y_test)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

total number of unique tags: 404 


In [24]:

parameters = [
    {
        'medoid_normalization':  NORMALIZATION,
        'svm_kernel': ['rbf','poly'],
        'svm_gamma': SVM_GAMMA,
        'svm_c':SVM_C,
        'svm_degree' :SVM_DEGREE,
        'vectorizer_norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    },
    {
        'medoid_normalization':  NORMALIZATION,
        'svm_kernel': ['linear'],
        'svm_gamma': SVM_GAMMA,
        'svm_c':SVM_C,
        'svm_degree' :[None],
        'vectorizer_norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    }
]




In [25]:
print('total number of configurations to test: {}'.format(len(ParameterGrid(parameters))))

total number of configurations to test: 4536


In [ ]:
best_score = float("-inf")
best_configuration = None

for (i,configuration) in enumerate(tqdm(ParameterGrid(parameters))):
    
    # TFIDF_VECTORIZER = COUNT_VECTORIZER + TFIDF_TRANSFORMER
    
    tfidf_vectorizer = TfidfVectorizer(max_features=MAX_NB_WORDS, norm=configuration['vectorizer_norm'])
    # TRAINING SET
    tfidf_vectorizer.fit(documents_train)
    tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)
        
    
    # THE FOLLOWING BLOCK TAKES SOME TIME, BUT IT WILL ONLY RUN ONCE
    
    path_to_cache = INTERIM_DATA_ROOT.rstrip('/') + "/mimlsvm/distance-matrix-train-sample-{}-{}-{}.p".format(
        NB_DOCS,
        MAX_NB_WORDS,
        configuration['vectorizer_norm'])
    
    if os.path.isfile(path_to_cache):
        dist_matrix_train = pickle.load(open(path_to_cache,"rb"))
    else:
        print('Fitting distance matrix for norm={}'.format(configuration['vectorizer_norm']))
        
        dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)
        pickle.dump(dist_matrix_train, open(path_to_cache, "wb"))
    
    # nb_medoids depends upon the dataset length
    ratio = configuration['nb_medoids_ratio']
    nb_medoids = int(len(tfidf_segments_train) * ratio)
    
    medoids_indices_train = k_medoids(dist_matrix_train,nb_medoids)[0]

    X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)
    
    # TEST SET
    
    tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)
          
    # medoids trained on the training set
    fitted_medoids = tfidf_segments_train[medoids_indices_train]
    X_test = make_test_dataset(tfidf_segments_test,fitted_medoids)
      
    clf = OneVsRestClassifier(
        SVC(kernel=configuration['svm_kernel'],
            gamma=configuration['svm_gamma'],
            C=configuration['svm_c'],
            degree=configuration['svm_degree'],
            probability=True,
           ),n_jobs=-1)    

   
    
    if configuration['medoid_normalization'] == 'standard':      
        scaler = StandardScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    elif configuration['medoid_normalization'] == 'minmax':
        scaler = MinMaxScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    else:
        X_train_final = X_train
        X_test_final = X_test
    
    # y_train was defined outside the loop    
    clf.fit(X_train,Y_train)
    
    # train score
    Y_pred_train = clf.predict_proba(X_train)
    
    # validation score
    Y_pred_test = clf.predict_proba(X_test)  
    
    print("iter: {}, configuration: {}\n".format(i,configuration))
    
    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
#         print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_test,Y_pred_test,k=k,normalize=True)))    

  0%|          | 0/4536 [00:00<?, ?it/s]

Fitting distance matrix for norm=l1


  0%|          | 1/4536 [05:30<416:36:52, 330.72s/it]

iter: 0, configuration: {'vectorizer_norm': 'l1', 'svm_degree': 3, 'medoid_normalization': 'standard', 'svm_kernel': 'rbf', 'nb_medoids_ratio': 0.2, 'svm_gamma': 0.1, 'svm_c': 0.001}

validation micro-F1 @1: 0.0049504950495049506
validation micro-F1 @2: 0.01485148514851485
validation micro-F1 @3: 0.009900990099009901
validation micro-F1 @4: 0.012376237623762377
validation micro-F1 @5: 0.01089108910891089
validation micro-F1 @6: 0.009900990099009901
validation micro-F1 @7: 0.009900990099009901
validation micro-F1 @8: 0.008663366336633664
validation micro-F1 @9: 0.00935093509350935
validation micro-F1 @10: 0.009900990099009901
Fitting distance matrix for norm=l2


  0%|          | 2/4536 [11:06<419:58:29, 333.46s/it]

...

iter: 680, configuration: {'vectorizer_norm': None, 'svm_degree': 3, 'medoid_normalization': 'standard', 'svm_kernel': 'rbf', 'nb_medoids_ratio': 0.3, 'svm_gamma': 'auto', 'svm_c': 0.1}

validation micro-F1 @1: 0.06930693069306931
validation micro-F1 @2: 0.0891089108910891
validation micro-F1 @3: 0.10066006600660066
validation micro-F1 @4: 0.10767326732673267
validation micro-F1 @5: 0.100990099009901
validation micro-F1 @6: 0.10231023102310231
validation micro-F1 @7: 0.09688826025459689
validation micro-F1 @8: 0.09467821782178218
validation micro-F1 @9: 0.09405940594059406
validation micro-F1 @10: 0.0900990099009901


 15%|█▌        | 682/4536 [22:58:23<129:49:21, 121.27s/it]

iter: 681, configuration: {'vectorizer_norm': 'l1', 'svm_degree': 3, 'medoid_normalization': 'standard', 'svm_kernel': 'poly', 'nb_medoids_ratio': 0.3, 'svm_gamma': 'auto', 'svm_c': 0.1}

validation micro-F1 @1: 0.009900990099009901
validation micro-F1 @2: 0.034653465346534656
validation micro-F1 @3: 0.05115511551155116
validation micro-F1 @4: 0.07301980198019802
validation micro-F1 @5: 0.07326732673267326
validation micro-F1 @6: 0.08168316831683169
validation micro-F1 @7: 0.0785007072135785
validation micro-F1 @8: 0.07797029702970297
validation micro-F1 @9: 0.08085808580858085
validation micro-F1 @10: 0.07673267326732673


 15%|█▌        | 683/4536 [22:59:26<129:41:50, 121.18s/it]

iter: 682, configuration: {'vectorizer_norm': 'l2', 'svm_degree': 3, 'medoid_normalization': 'standard', 'svm_kernel': 'poly', 'nb_medoids_ratio': 0.3, 'svm_gamma': 'auto', 'svm_c': 0.1}

validation micro-F1 @1: 0.13861386138613863
validation micro-F1 @2: 0.12623762376237624
validation micro-F1 @3: 0.13036303630363036
validation micro-F1 @4: 0.12623762376237624
validation micro-F1 @5: 0.11782178217821782
validation micro-F1 @6: 0.10973597359735973
validation micro-F1 @7: 0.10678925035360678
validation micro-F1 @8: 0.10148514851485149
validation micro-F1 @9: 0.10121012101210121
validation micro-F1 @10: 0.09603960396039604
